In [15]:
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [16]:
data_text = "Juana. esclava Domingo veinte y dos de y nueve yo Thomas de Orvera baptize, y pusse santos oleos a Juana de nacion Mina esclava de Juan Joseph de Justis fueron sus P.P. Joseph Salcedo y Ana de Santiago su mugger, y lo firmé. Thomas de Orvera Paula. esclava Juebes veinte y tres de febrero de mil setecientos. y diez y nueve Yo Thomas de Orvera baptizé, y pusse los santos15 oleos á Paula h. l.16 de Juan Joseph, y Maria Josepha esclavos del Capitán Don Luis Hurtado de Mendoza fue su padrino Bartholome Rixo, y lo firmé. Thomas de Orvera Maria esclava Miercoles primero de febrero de mil sietectos y diez y nueve Yo Thomas de Orvera baptizé, y pusse los santos oleos á Maria h. l. de Juan, y Josepha esclavos del Capitán Antonio Benites fue su Madrina Ysabel Mendez, y lo firmé. Thomas de Orvera Bernardo esclavo Domingo nueve de Abril de mil setecientos y diez y nueve Yo Thomas de Orvera baptize, y pusse los santos oleos á Bernardo negro adulto de nacion Carabalí esclavo de Don Juan Joseph de Justis fue su Padre. Andres de Morales, y lo firmé. Thomas de Orvera Francisco esclavo Abril de mil setecientos y diez, y nueve o Thmas de vera bapizé, y pusse los s.tos oleo a Francisco negro adulto de nacion temo esclavo de Don Ju Joseph de Justis fue su Padre. Pedro Suares, y lo firmé. Thomas de Orvera Antonio esclavo Domingo nueve de Abril de mil setecientos y diez y nueve Yo Thomas de Orvera baptize, y pusse los santos oleos á Antonio negro adulto de nacion Carabalí esclavo de Don Juan Joseph de Justis fue su Padre. Joseph de Soto, y lo firmé. Thomas de Orvera Antonia esclava Domingo nueve de Abril de mil setecientos y diez, y nueve Yo Thomas de Orvera baptize, y pusse los santos oleos á Antonia negra adulta de nacion Mina esclava de Don Juan Joseph de Justis fue su Padre. Joseph Salcedo, y lo firmé. Thomas de Orvera Maria Luisa esclava Domingo nueve de Abril de mil setecientos y diez, y nueve Yo Thomas de Orvera baptize, y puse los santos oleos á Maria Luisa de nacion Lucumí negra adulta esclava de Don Juan Joseph de Justis fue su Padre. Jacinto de Castro, y lo firmé."

In [17]:
import re

def text_cleaner(text):
    # lower case text
    newString = text.lower()
    newString = re.sub(r"'s\b","",newString)
    # remove punctuations
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    long_words=[]
    # remove short word
    for i in newString.split():
        if len(i)>=3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

# preprocess the text
data_new = text_cleaner(data_text)

In [18]:
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# create sequences   
sequences = create_seq(data_new)

Total Sequences: 1655


In [19]:
# create a character mapping index
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        # integer encode line
        encoded_seq = [mapping[char] for char in line]
        # store
        sequences.append(encoded_seq)
    return sequences

# encode the sequences
sequences = encode_seq(sequences)

In [20]:
from sklearn.model_selection import train_test_split

# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (1489, 30) Val shape: (166, 30)


In [23]:
# define model
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# fit the model
model.fit(X_tr, y_tr, epochs=20, verbose=2, validation_data=(X_val, y_val))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 30, 50)            1200      
                                                                 
 gru_3 (GRU)                 (None, 150)               90900     
                                                                 
 dense_3 (Dense)             (None, 24)                3624      
                                                                 
Total params: 95,724
Trainable params: 95,724
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/20
47/47 - 8s - loss: 2.9709 - acc: 0.1444 - val_loss: 2.9246 - val_acc: 0.1687 - 8s/epoch - 171ms/step
Epoch 2/20
47/47 - 4s - loss: 2.7723 - acc: 0.1713 - val_loss: 2.7626 - val_acc: 0.2410 - 4s/epoch - 84ms/step
Epoch 3/20
47/47 - 5s - loss: 2.5745 - acc: 0.2626 - val_loss: 2.4657 - val_acc: 0.289

In [26]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        # predict character
        yhat = np.argmax(model.predict(encoded), axis=-1)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += out_char
    return in_text

In [28]:
print(generate_seq(model, mapping, 30, 'juana esclava', 100))

ValueError: ignored